In [2]:
def detect_stress(alpha_power, beta_power, threshold=1.0):
    alpha_beta_ratio = alpha_power / beta_power
    if alpha_beta_ratio > threshold:
        return "Relaxed"
    else:
        return "Stressed"



In [3]:
from pylsl import StreamInlet, resolve_byprop

print("Looking for EEG streams...")
streams = resolve_byprop('type', 'EEG')
inlet = StreamInlet(streams[0])

Looking for EEG streams...


2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2025-03-04 20:21:52.577 (   0.004s) [          79FD54]      netinterfaces.cpp:91    I

In [4]:
import numpy as np
from scipy.signal import welch

def real_time_alpha_beta(stream, chunk_size=256, sampling_rate=256):
    """
    Extract Alpha and Beta power in real-time from an EEG stream.
    :param stream: EEG data stream (e.g., from Muse LSL).
    :param chunk_size: Number of samples to process at a time.
    :param sampling_rate: Sampling rate of the EEG data.
    """
    buffer = np.zeros(chunk_size)
    index = 0

    while True:
        # Get new data from the stream
        new_data = stream.get_data()  # Replace with actual stream reading logic
        buffer[index:index + len(new_data)] = new_data
        index += len(new_data)

        if index >= chunk_size:
            # Process the chunk
            alpha_power, beta_power = extract_alpha_beta_power(buffer, sampling_rate)
            print(f"Alpha Power: {alpha_power}, Beta Power: {beta_power}")

            # Reset the buffer
            buffer = np.zeros(chunk_size)
            index = 0

In [ ]:
import numpy as np
from scipy.signal import welch
import muselsl
from pylsl import StreamInlet, resolve_byprop

def extract_alpha_beta_power(eeg_data, sampling_rate=256):
    """
    Extract Alpha and Beta power from EEG data.
    :param eeg_data: Raw EEG data (1D array).
    :param sampling_rate: Sampling rate of the EEG data (default 256 Hz for Muse).
    :return: Alpha power, Beta power.
    """
    # Compute the Power Spectral Density (PSD) using Welch's method
    freqs, psd = welch(eeg_data, fs=sampling_rate, nperseg=1024)

    # Define frequency bands
    alpha_band = (8, 12)  # Alpha waves
    beta_band = (12, 30)  # Beta waves

    # Extract power in the Alpha and Beta bands
    alpha_power = np.mean(psd[(freqs >= alpha_band[0]) & (freqs <= alpha_band[1])])
    beta_power = np.mean(psd[(freqs >= beta_band[0]) & (freqs <= beta_band[1])])

    return alpha_power, beta_power

def detect_stress(alpha_power, beta_power, threshold=1.0):
    """
    Detect stress based on Alpha/Beta ratio.
    :param alpha_power: Alpha power value.
    :param beta_power: Beta power value.
    :param threshold: Threshold for stress detection.
    :return: "Stressed" or "Relaxed".
    """
    alpha_beta_ratio = alpha_power / beta_power
    if alpha_beta_ratio > threshold:
        return "Relaxed"
    else:
        return "Stressed"

def real_time_alpha_beta():
    """
    Stream EEG data from Muse S Gen 2 and extract Alpha/Beta power in real-time.
    """
    # Resolve the EEG stream
    print("Looking for an EEG stream...")
    streams = resolve_byprop('type', 'EEG')
    inlet = StreamInlet(streams[0])  # Create an inlet to read data

    print("Stream found. Starting real-time processing...")
    chunk_size = 256  # Process data in chunks of 256 samples
    sampling_rate = 256  # Muse S Gen 2 sampling rate
    buffer = np.zeros(chunk_size)
    index = 0

    while True:
        # Get a new sample from the stream
        sample, _ = inlet.pull_sample()  # sample contains [channel1, channel2, channel3, channel4]
        eeg_sample = sample[0]  # Use the first channel (FP1) for simplicity

        # Add the sample to the buffer
        buffer[index] = eeg_sample
        index += 1

        # Process the chunk when the buffer is full
        if index >= chunk_size:
            # Extract Alpha and Beta power
            alpha_power, beta_power = extract_alpha_beta_power(buffer, sampling_rate)

            # Detect stress
            stress_status = detect_stress(alpha_power, beta_power, threshold=1.0)
            print(f"Alpha Power: {alpha_power:.4f}, Beta Power: {beta_power:.4f}, Status: {stress_status}")

            # Reset the buffer
            buffer = np.zeros(chunk_size)
            index = 0

if __name__ == "__main__":
    real_time_alpha_beta()

Looking for an EEG stream...
Stream found. Starting real-time processing...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/signal/_spectral_py.py:589: UserWarning: nperseg = 1024 is greater than input length  = 256, using nperseg = 256
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Alpha Power: 7.6810, Beta Power: 1.8349, Status: Relaxed
Alpha Power: 3.2620, Beta Power: 1.5472, Status: Relaxed
Alpha Power: 1.8294, Beta Power: 1.6171, Status: Relaxed
Alpha Power: 0.5503, Beta Power: 0.9694, Status: Stressed
Alpha Power: 2.1712, Beta Power: 1.0121, Status: Relaxed
Alpha Power: 2.1038, Beta Power: 0.8830, Status: Relaxed
Alpha Power: 25.0077, Beta Power: 3.2432, Status: Relaxed
Alpha Power: 7.5691, Beta Power: 2.7823, Status: Relaxed
Alpha Power: 552.1477, Beta Power: 34.4380, Status: Relaxed
Alpha Power: 7.0271, Beta Power: 4.3821, Status: Relaxed
Alpha Power: 6.5512, Beta Power: 3.6920, Status: Relaxed
Alpha Power: 435.8767, Beta Power: 80.9228, Status: Relaxed
Alpha Power: 141.4695, Beta Power: 31.2452, Status: Relaxed
Alpha Power: 927.7029, Beta Power: 87.9318, Status: Relaxed
Alpha Power: 47.0068, Beta Power: 7.6510, Status: Relaxed
Alpha Power: 10.9329, Beta Power: 2.8545, Status: Relaxed
Alpha Power: 23.5478, Beta Power: 2.3853, Status: Relaxed
Alpha Power: 2

2025-03-04 20:56:30.393 (2077.810s) [R_Muse          ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...


Alpha Power: 2.1898, Beta Power: 1.1993, Status: Relaxed
Alpha Power: 1.5026, Beta Power: 1.6055, Status: Stressed
Alpha Power: 2.0708, Beta Power: 1.1077, Status: Relaxed
Alpha Power: 15.4714, Beta Power: 0.9447, Status: Relaxed
Alpha Power: 8.6282, Beta Power: 1.2840, Status: Relaxed
Alpha Power: 2.3478, Beta Power: 1.8140, Status: Relaxed
Alpha Power: 21.5217, Beta Power: 3.4021, Status: Relaxed
Alpha Power: 3.6936, Beta Power: 1.7896, Status: Relaxed
Alpha Power: 1.9891, Beta Power: 1.5189, Status: Relaxed
Alpha Power: 4.3128, Beta Power: 0.9985, Status: Relaxed
Alpha Power: 7.9191, Beta Power: 1.4609, Status: Relaxed
Alpha Power: 4.6940, Beta Power: 3.0451, Status: Relaxed
Alpha Power: 6.2774, Beta Power: 1.3304, Status: Relaxed
Alpha Power: 13.7694, Beta Power: 2.1321, Status: Relaxed
Alpha Power: 13.7336, Beta Power: 1.1269, Status: Relaxed
Alpha Power: 46.4439, Beta Power: 2.7022, Status: Relaxed
Alpha Power: 17.1161, Beta Power: 2.4385, Status: Relaxed
Alpha Power: 7.2662, Bet

2025-03-04 21:20:01.100 (3488.515s) [R_Muse          ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...


Alpha Power: 342.6104, Beta Power: 62.5294, Status: Relaxed
Alpha Power: 7.1516, Beta Power: 3.1133, Status: Relaxed
Alpha Power: 2.2504, Beta Power: 2.3402, Status: Stressed
Alpha Power: 7.4942, Beta Power: 2.3636, Status: Relaxed
Alpha Power: 35.5637, Beta Power: 4.8871, Status: Relaxed
Alpha Power: 12.5287, Beta Power: 2.9353, Status: Relaxed
Alpha Power: 2.5738, Beta Power: 2.2680, Status: Relaxed
Alpha Power: 5.8881, Beta Power: 1.7736, Status: Relaxed
Alpha Power: 15.8172, Beta Power: 1.3094, Status: Relaxed
Alpha Power: 6.2373, Beta Power: 1.5558, Status: Relaxed
Alpha Power: 15.7764, Beta Power: 2.0826, Status: Relaxed
Alpha Power: 1.5811, Beta Power: 2.0003, Status: Stressed
Alpha Power: 7.4023, Beta Power: 2.8039, Status: Relaxed
Alpha Power: 7.2905, Beta Power: 1.1043, Status: Relaxed
Alpha Power: 2.1684, Beta Power: 3.3776, Status: Stressed
Alpha Power: 2.7361, Beta Power: 2.2207, Status: Relaxed
Alpha Power: 6.4844, Beta Power: 2.6221, Status: Relaxed
Alpha Power: 3.3758, 

2025-03-04 21:41:48.646 (4796.059s) [R_Muse          ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
